In [25]:

import pandas as pd
import datetime
import statsmodels.tsa.stattools as ts
import os 
from pandas_datareader import data as pdr

In [29]:
#Arrays donde vamos a guardar los tickers que cumplan mean reversion al 99% o al 95%
mr_arr99 = []
mr_arr95 = [] 
mr_arr90 = []

#Los tickers del S&P500
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0] 
ticker_list=df['Symbol'].tolist() 

#Elegimos fechas sobre los que haremos los estudios de las series 
start = datetime.datetime.now() - datetime.timedelta(days=365)
end = datetime.datetime.now()

In [32]:
#Comprobamos ADF por cada ticker, y vamos guardando en los array 
for ticker in ticker_list: 
    try:        
        df= pdr.get_data_yahoo(ticker,start= start, end= end) 
        print(f'Analizando {ticker}...')
        df=df['Close'] 
        result= ts.adfuller(df,1) 
        print(result)
        # Añadimos los tickers a los arrays si pasan el test de Augmented Dickey Fuller 
        if result[0] < result[4]['1%']: 
            mr_arr99.append(ticker)
        elif result[0] < result[4]['5%']: 
            mr_arr95.append(ticker)
        elif result[0] < result[4]['10%']:  
            mr_arr90.append(ticker)
        else: 
            print(f'Ticker {ticker} no ha pasado el filtro') 
    except: 
        pass






56780859712, '5%': -2.8731715065600003, '10%': -2.572968544}, 1335.2515455945306)
Ticker RCL no ha pasado el filtro
Analizando SPGI...
(-1.3186757580218957, 0.6205854130487684, 1, 250, {'1%': -3.456780859712, '5%': -2.8731715065600003, '10%': -2.572968544}, 1699.5668096638271)
Ticker SPGI no ha pasado el filtro
Analizando CRM...
(-0.8941537668507518, 0.7898915688084804, 0, 251, {'1%': -3.4566744514553016, '5%': -2.8731248767783426, '10%': -2.5729436702592023}, 1604.0067854929086)
Ticker CRM no ha pasado el filtro
Analizando SBAC...
(-1.9141645148104838, 0.3254272955249513, 1, 250, {'1%': -3.456780859712, '5%': -2.8731715065600003, '10%': -2.572968544}, 1703.1654457306536)
Ticker SBAC no ha pasado el filtro
Analizando SLB...
(-0.9548775366728423, 0.769364064419181, 0, 251, {'1%': -3.4566744514553016, '5%': -2.8731248767783426, '10%': -2.5729436702592023}, 669.159714682987)
Ticker SLB no ha pasado el filtro
Analizando STX...
(-1.6828183114311646, 0.4399860311690085, 1, 250, {'1%': -3.456

In [33]:
print(mr_arr90)

['BIIB', 'CERN', 'CMS', 'DLTR', 'EXPE', 'HRL', 'KMB', 'LMT', 'MCK', 'TIF']


In [34]:
print(mr_arr95)

['BAX', 'BAX', 'CNC', 'DVA', 'D', 'ES', 'SJM', 'LUMN', 'WEC']


In [35]:
print(mr_arr99)

['ANET', 'ANET', 'CPB', 'K']
